# EMG分类器详解与对比

## 教学内容

### 监督学习分类器
1. **SVM** (Support Vector Machine) - 支持向量机
2. **KNN** (K-Nearest Neighbors) - K近邻
3. **Random Forest** - 随机森林
4. **简单神经网络** (MLP)

### 无监督学习
5. **K-Means聚类**
6. **层次聚类**

### 对比分析
- 准确率对比
- 训练时间对比
- 参数敏感性
- 适用场景

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import time

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from code.week07_feature_extraction.features import EMGFeatures
from code.week06_preprocessing.filters import EMGFilters

plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

print("✓ 导入成功！")

## 准备数据集

生成三种手势的EMG数据用于分类。

In [ ]:
def generate_gesture_data(gesture, n_samples=30):
    """生成指定手势的数据"""
    fs = 1000
    filters = EMGFilters(fs=fs)
    samples = []
    
    for _ in range(n_samples):
        t = np.linspace(0, 2, 2000)
        signal = np.random.normal(0, 0.02, len(t))
        
        if gesture == 'rest':
            # 静息：只有噪声
            pass
        elif gesture == 'fist':
            # 握拳：强收缩
            for freq in range(70, 140, 15):
                signal += 0.5 * np.sin(2 * np.pi * freq * t)
        elif gesture == 'open':
            # 张开：中等收缩
            for freq in range(60, 120, 15):
                signal += 0.3 * np.sin(2 * np.pi * freq * t)
        
        signal_clean = filters.preprocess_emg(signal, remove_powerline=True)
        time_feat = EMGFeatures.extract_time_features(signal_clean)
        freq_feat = EMGFeatures.extract_freq_features(signal_clean, fs=fs)
        
        sample = {**time_feat, **freq_feat, 'gesture': gesture}
        samples.append(sample)
    
    return samples

# 生成数据
data = []
gestures = ['rest', 'fist', 'open']

for gesture in gestures:
    data.extend(generate_gesture_data(gesture, n_samples=30))

df = pd.DataFrame(data)
print(f"✓ 数据生成完成：{len(df)}个样本")
print(f"  每种手势: 30个样本")

In [ ]:
# 准备训练数据
X = df.drop('gesture', axis=1).values
y = df['gesture'].values

# 标签编码
label_map = {'rest': 0, 'fist': 1, 'open': 2}
y_numeric = np.array([label_map[g] for g in y])

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(
    X, y_numeric, test_size=0.3, random_state=42, stratify=y_numeric
)

# 归一化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"训练集: {X_train.shape}")
print(f"测试集: {X_test.shape}")

---
## 1. SVM (支持向量机)

### 原理
- 寻找最优超平面分隔不同类别
- 使用核函数处理非线性问题
- 只关注边界附近的样本（支持向量）

### 优点
- 高维数据表现好
- 泛化能力强
- 理论基础扎实

### 缺点
- 训练较慢（大数据集）
- 参数调优复杂
- 难以解释

In [ ]:
# 训练SVM
print("训练SVM...")
start_time = time.time()

svm = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm.fit(X_train_scaled, y_train)
y_pred_svm = svm.predict(X_test_scaled)
acc_svm = accuracy_score(y_test, y_pred_svm)

train_time_svm = time.time() - start_time

print(f"  准确率: {acc_svm:.2%}")
print(f"  训练时间: {train_time_svm:.4f}秒")
print(f"  支持向量数: {len(svm.support_)}")

---
## 2. KNN (K近邻)

### 原理
- 找到最近的k个邻居
- 通过投票决定类别
- 简单直观，无需训练

### 优点
- 实现简单
- 无需训练过程
- 对异常值鲁棒

### 缺点
- 预测速度慢
- 需要存储所有训练数据
- 对特征缩放敏感

In [ ]:
# 训练KNN
print("训练KNN...")
start_time = time.time()

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
y_pred_knn = knn.predict(X_test_scaled)
acc_knn = accuracy_score(y_test, y_pred_knn)

train_time_knn = time.time() - start_time

print(f"  准确率: {acc_knn:.2%}")
print(f"  训练时间: {train_time_knn:.4f}秒")
print(f"  k值: 5")

---
## 3. Random Forest (随机森林)

### 原理
- 训练多个决策树
- 每棵树使用随机的特征子集
- 投票决定最终结果

### 优点
- 准确率高
- 不易过拟合
- 可以评估特征重要性
- 对参数不敏感

### 缺点
- 模型较大
- 可解释性一般

In [ ]:
# 训练Random Forest
print("训练Random Forest...")
start_time = time.time()

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)
acc_rf = accuracy_score(y_test, y_pred_rf)

train_time_rf = time.time() - start_time

print(f"  准确率: {acc_rf:.2%}")
print(f"  训练时间: {train_time_rf:.4f}秒")
print(f"  树的数量: 100")

---
## 4. MLP神经网络（最简单的深度学习）

### 网络结构
```
输入层 (18个特征)
    ↓
隐藏层1 (64个神经元) + ReLU
    ↓
隐藏层2 (32个神经元) + ReLU
    ↓
输出层 (3个类别) + Softmax
```

### 优点
- 可以学习复杂的非线性关系
- 适合大数据集
- 可扩展性强

### 缺点
- 需要更多训练数据
- 训练时间较长
- 超参数多

In [ ]:
# 训练简单的MLP神经网络
print("训练MLP神经网络...")
start_time = time.time()

mlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),  # 两个隐藏层
    activation='relu',
    max_iter=1000,
    random_state=42
)

mlp.fit(X_train_scaled, y_train)
y_pred_mlp = mlp.predict(X_test_scaled)
acc_mlp = accuracy_score(y_test, y_pred_mlp)

train_time_mlp = time.time() - start_time

print(f"  准确率: {acc_mlp:.2%}")
print(f"  训练时间: {train_time_mlp:.4f}秒")
print(f"  网络结构: 输入{X_train.shape[1]} → 64 → 32 → 输出3")
print(f"  迭代次数: {mlp.n_iter_}")

---
## 监督学习分类器对比

In [ ]:
# 汇总结果
results = pd.DataFrame([
    {'分类器': 'SVM', '准确率': acc_svm * 100, '训练时间(秒)': train_time_svm},
    {'分类器': 'KNN', '准确率': acc_knn * 100, '训练时间(秒)': train_time_knn},
    {'分类器': 'Random Forest', '准确率': acc_rf * 100, '训练时间(秒)': train_time_rf},
    {'分类器': 'MLP神经网络', '准确率': acc_mlp * 100, '训练时间(秒)': train_time_mlp}
])

print("\n分类器性能对比：")
print("=" * 60)
print(results.to_string(index=False))

# 可视化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 准确率对比
colors = ['#3498db', '#e74c3c', '#2ecc71', '#9b59b6']
bars1 = ax1.bar(results['分类器'], results['准确率'], color=colors, alpha=0.8)
ax1.set_ylabel('准确率 (%)', fontsize=12)
ax1.set_title('准确率对比', fontsize=13, fontweight='bold')
ax1.set_ylim(0, 100)
ax1.grid(True, alpha=0.3, axis='y')

for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}%', ha='center', va='bottom', fontsize=10, fontweight='bold')

# 训练时间对比
bars2 = ax2.bar(results['分类器'], results['训练时间(秒)'], color=colors, alpha=0.8)
ax2.set_ylabel('训练时间 (秒)', fontsize=12)
ax2.set_title('训练时间对比', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.3f}s', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

---
## 5. K-Means聚类（无监督学习）

### 什么是聚类？
- **不需要标签**的机器学习
- 自动发现数据中的**群组**
- 用于**探索性数据分析**

### K-Means原理
1. 随机初始化k个中心点
2. 将每个样本分配到最近的中心
3. 更新中心点位置
4. 重复2-3直到收敛

In [ ]:
# K-Means聚类
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_train_scaled)

print("K-Means聚类结果：")
print(f"  聚类数: 3")
print(f"  迭代次数: {kmeans.n_iter_}")
print(f"\n各簇样本数:")
unique, counts = np.unique(clusters, return_counts=True)
for cluster_id, count in zip(unique, counts):
    print(f"  簇{cluster_id}: {count}个样本")

# 对比真实标签和聚类结果
print(f"\n注意：聚类是无监督的，簇的编号不一定对应真实标签")

In [ ]:
# 可视化聚类结果（使用PCA降到2D）
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_train_scaled)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 真实标签
scatter1 = ax1.scatter(X_pca[:, 0], X_pca[:, 1], c=y_train, cmap='viridis', alpha=0.6, s=50)
ax1.set_xlabel('主成分1', fontsize=11)
ax1.set_ylabel('主成分2', fontsize=11)
ax1.set_title('真实标签', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=ax1, label='手势类别')

# 聚类结果
scatter2 = ax2.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='Set1', alpha=0.6, s=50)
ax2.set_xlabel('主成分1', fontsize=11)
ax2.set_ylabel('主成分2', fontsize=11)
ax2.set_title('K-Means聚类结果', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=ax2, label='簇编号')

plt.tight_layout()
plt.show()

print("\n说明：PCA将18维特征降到2维用于可视化")

---
## 最终对比总结

In [ ]:
# 详细对比表
comparison = pd.DataFrame([
    {
        '分类器': 'SVM',
        '准确率': f"{acc_svm:.2%}",
        '训练时间': f"{train_time_svm:.4f}s",
        '优点': '高维数据好，泛化强',
        '缺点': '训练慢，参数多',
        '推荐场景': '特征多，数据中等'
    },
    {
        '分类器': 'KNN',
        '准确率': f"{acc_knn:.2%}",
        '训练时间': f"{train_time_knn:.4f}s",
        '优点': '简单直观，无需训练',
        '缺点': '预测慢，内存占用大',
        '推荐场景': '小数据集，快速原型'
    },
    {
        '分类器': 'Random Forest',
        '准确率': f"{acc_rf:.2%}",
        '训练时间': f"{train_time_rf:.4f}s",
        '优点': '准确率高，稳定',
        '缺点': '模型大',
        '推荐场景': '大多数EMG应用（推荐）'
    },
    {
        '分类器': 'MLP神经网络',
        '准确率': f"{acc_mlp:.2%}",
        '训练时间': f"{train_time_mlp:.4f}s",
        '优点': '学习复杂模式',
        '缺点': '需要更多数据',
        '推荐场景': '大数据集，复杂任务'
    }
])

print("\n" + "="*80)
print("分类器详细对比".center(80))
print("="*80)
print(comparison.to_string(index=False))
print("="*80)

### 混淆矩阵对比

In [ ]:
# 绘制四个分类器的混淆矩阵
predictions = [
    ('SVM', y_pred_svm),
    ('KNN', y_pred_knn),
    ('Random Forest', y_pred_rf),
    ('MLP', y_pred_mlp)
]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('混淆矩阵对比', fontsize=16, fontweight='bold')

gesture_names = ['rest', 'fist', 'open']

for idx, (name, y_pred) in enumerate(predictions):
    row = idx // 2
    col = idx % 2
    
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=gesture_names,
                yticklabels=gesture_names,
                ax=axes[row, col])
    
    axes[row, col].set_title(name, fontsize=12, fontweight='bold')
    axes[row, col].set_xlabel('预测标签')
    axes[row, col].set_ylabel('真实标签')

plt.tight_layout()
plt.show()

---
## 选择建议

### 根据数据量选择
- **小数据集**（< 100样本）：KNN 或 SVM
- **中等数据集**（100-1000样本）：Random Forest（推荐）
- **大数据集**（> 1000样本）：Random Forest 或 MLP

### 根据应用场景选择
- **实时系统**：Random Forest（预测快）
- **离线分析**：任意方法
- **嵌入式设备**：KNN 或简单的决策树
- **研究项目**：对比多种方法

### 推荐的学习顺序
1. 先学**KNN**（最简单，理解分类原理）
2. 再学**Random Forest**（最实用）
3. 然后学**SVM**（理论深入）
4. 最后学**神经网络**（高级应用）